In [6]:
!pip install faker graphviz
import sqlite3
import random

conn = sqlite3.connect("cinema_theater.db")
cursor = conn.cursor()

cursor.executescript("""
DROP TABLE IF EXISTS Tickets;
DROP TABLE IF EXISTS Showtimes;
DROP TABLE IF EXISTS Customers;
DROP TABLE IF EXISTS Movies;
DROP TABLE IF EXISTS Theaters;
""")

cursor.execute("""
CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    membership_level TEXT CHECK(membership_level IN ('Bronze','Silver','Gold','Platinum')),
    age INTEGER CHECK(age >= 0)
)
""")

cursor.execute("""
CREATE TABLE Movies (
    movie_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    genre TEXT,
    duration_minutes INTEGER CHECK(duration_minutes > 0),
    release_year INTEGER
)
""")

cursor.execute("""
CREATE TABLE Theaters (
    theater_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    capacity INTEGER CHECK(capacity > 0),
    location TEXT
)
""")

cursor.execute("""
CREATE TABLE Showtimes (
    showtime_id INTEGER PRIMARY KEY AUTOINCREMENT,
    movie_id INTEGER,
    theater_id INTEGER,
    show_date DATE,
    start_time TIME,
    FOREIGN KEY(movie_id) REFERENCES Movies(movie_id),
    FOREIGN KEY(theater_id) REFERENCES Theaters(theater_id)
)
""")

cursor.execute("""
CREATE TABLE Tickets (
    ticket_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    showtime_id INTEGER,
    seat_number TEXT,
    price REAL CHECK(price >= 0),
    FOREIGN KEY(customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY(showtime_id) REFERENCES Showtimes(showtime_id)
)
""")

membership_levels = ["Bronze", "Silver", "Gold", "Platinum"]
genres = ["Action", "Comedy", "Drama", "Horror", "Sci-Fi", "Romance"]

for _ in range(300):
    cursor.execute("INSERT INTO Customers (name, email, membership_level, age) VALUES (?, ?, ?, ?)",
                   (fake.name(), fake.unique.email(), random.choice(membership_levels), random.randint(12, 80)))

for _ in range(50):
    cursor.execute("INSERT INTO Movies (title, genre, duration_minutes, release_year) VALUES (?, ?, ?, ?)",
                   (fake.catch_phrase(), random.choice(genres), random.randint(80, 180), random.randint(1990, 2025)))

for i in range(10):
    cursor.execute("INSERT INTO Theaters (name, capacity, location) VALUES (?, ?, ?)",
                   (f"Theater {i+1}", random.randint(100, 300), fake.city()))

movie_ids = [row[0] for row in cursor.execute("SELECT movie_id FROM Movies")]
theater_ids = [row[0] for row in cursor.execute("SELECT theater_id FROM Theaters")]

for _ in range(200):
    cursor.execute("INSERT INTO Showtimes (movie_id, theater_id, show_date, start_time) VALUES (?, ?, ?, ?)",
                   (random.choice(movie_ids), random.choice(theater_ids),
                    fake.date_between(start_date="-30d", end_date="+30d").isoformat(), fake.time()))

customer_ids = [row[0] for row in cursor.execute("SELECT customer_id FROM Customers")]
showtime_ids = [row[0] for row in cursor.execute("SELECT showtime_id FROM Showtimes")]

for _ in range(1000):
    cursor.execute("INSERT INTO Tickets (customer_id, showtime_id, seat_number, price) VALUES (?, ?, ?, ?)",
                   (random.choice(customer_ids), random.choice(showtime_ids),
                    f"{random.choice('ABCDE')}{random.randint(1,30)}", round(random.uniform(5.0, 20.0), 2)))

conn.commit()
conn.close()


